In [3]:
#I'm learning how others use pipelines and cv. Special thanks to Jaafar SAADANI
#https://www.kaggle.com/jsaadani/simple-ridge-regression-in-python/code
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib
import itertools
from scipy.stats import skew
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from scipy.stats.stats import pearsonr
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)

%matplotlib inline

In [94]:
train = pd.read_csv("input/train.csv") # no missing data
test = pd.read_csv("input/test.csv")

In [95]:
allData = pd.get_dummies(train.append(test))
train = allData[:len(train)]
train = train.drop("ID", axis=1)
test = allData[len(train):]
ids = test.ID
test = test.drop(["ID","y"], axis=1)

In [96]:
X_train = train.loc[:, train.columns != 'y']
y_train = train.y
X_test = test

In [97]:
"""from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.loc[:, train.columns != 'y'], train.y, test_size=0.2, random_state=1)
X_train.shape"""

"from sklearn.model_selection import train_test_split\nX_train, X_test, y_train, y_test = train_test_split(train.loc[:, train.columns != 'y'], train.y, test_size=0.2, random_state=1)\nX_train.shape"

In [105]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# steps
steps = [('scaler', StandardScaler()),
         ('ridge', Ridge())]

# Create the pipeline: pipeline
pipeline = Pipeline(steps)

# Specify the hyperparameter space
parameters = {'ridge__alpha':np.logspace(-10, 0, 50)}

# Create the GridSearchCV object: cv
cv = GridSearchCV(pipeline, parameters, cv=3)

# Fit to the training set
cv.fit(X_train, y_train)

#predict on train set
y_pred_train=cv.predict(X_train)

# Predict test set
y_pred_test=cv.predict(X_test)

# rmse on train set
rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
print("Root Mean Squared Error: {}".format(rmse))

# shape to export
#output=pd.concat([y_id, DataFrame(np.exp(y_pred_test)-1)], axis=1, ignore_index=True)
#output.columns=['Id', 'SalePrice']

# export
#output.to_csv('./submission.csv', sep=',', index=False)

/home/splacorn/miniconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 9.138556108688294e-17 / 1.1102230246251565e-16
  RuntimeWarning)


Root Mean Squared Error: 7.815733942391403


In [102]:
my_submission = pd.DataFrame({'ID': ids, 'y': y_pred_test})
# you could use any filename. We choose submission here
#my_submission.to_csv('submissions/submission1.csv', index=False)